In [1]:
import sys
import docplex.mp as mp
from docplex.mp.model import Model

In [4]:
class OR2_Integer_Project:
    def __init__(self):
        self.model=Model(name='OR2 Integer Project')
        
        #Set type of Candies and Worker Types
        candy_name=['Fruity Fun',"Chocolate Craze","Tis the Season"]
        worker_job=["Line Worker", "Technician", "Supervisor", "Engineer"]
        
        #Demand Requirements in Arrays
        plant1_demand= [[50,58,40,35,30,37,45,48,35,36,40,44],
                        [6,8,5,5,4,5,6,6,7,8,7,6],
                        [10,9,10,12,15,15,10,8,9,9,16,12],
                        [7,9,6,8,6,5,8,7,6,7,6,4]]
        plant2_demand= [[30,48,40,35,33,27,38,38,32,26,45,42],
                        [6,6,7,5,7,5,6,5,6,7,6,4],
                        [13,15,12,12,11,11,10,8,9,10,12,12],
                        [8,8,7,9,6,6,7,6,5,5,6,3]]
        plant3_demand= [[40,48,40,35,33,27,38,38,32,26,45,42],
                        [6,6,7,5,7,5,6,5,6,7,6,4],
                        [15,15,15,13,11,15,12,9,9,11,11,10],
                        [7,8,8,8,7,6,8,8,6,8,5,5]]
        plant4_demand= [[50,55,45,45,40,32,35,42,30,32,47,40],
                        [6,6,6,5,4,7,7,7,9,8,7,7],
                        [12,12,13,12,11,15,8,9,10,11,11,12],
                        [7,8,7,6,6,9,8,7,7,7,7,6]]
        
        cost_of_workers=[4200,4800,5000,6600]
        
        cost_of_maintaining=[[35000,40000,41000],
                             [28000,30000,27000],
                             [25000,27000,28000],
                             [30000,35000,29000]]
        
        #Value of each 
        months = 12
        candys = 3
        plants = 4
        workers = 4
        
    def create_variables(self):
        self.assigned = []
        
        for _ in range(plants):
            plant_assignment = []
            for _ in range(workers):
                worker_type_assignment = []
                for _ in range(months):
                    worker_type_assignment.append(self.model.integer_var())
                plant_assignment.append(worker_type_assignment)
            self.assigned.append(plant_assignment)
            
        
        self.plant_produce_candy_y = []
        
        for _ in range(plants):
            plant_produce = []
            for _ in range(candys):
                plant_produce.append(self.model.binary_var())
            self.plant_produce_candy_y.append(plant_produce)
            
             
    def create_objective(self):
        self.model.minimize(sum(self.assigned[i][j][k]*cost_of_workers[i] for i in range(workers) for j in range(plants) for k in range(month)),
                            sum(self.plant_produce_candy_y[j][y] * cost_of_maintaining[j][y] for j in range(plants) for y in range(candys)))
        
    def create_constraints(self): 
        #each candy has to be made at minimum 2 plants
        for y in range(candys):
            self.model.add_constrant(self.plant_produce_candy_y[0][y] +self.plant_produce_candy_y[1][y]
                                     + self.plant_produce_candy_y[2][y] +self.plant_produce_candy_y[3][y] >= 2)
        
        #Constraints Meeting Demand for Workers 
        for k in range(months):
            for i in range(workers):
                for y in range(candys): 
                    self.model.add_constraint(plant1_demand[i][k] * self.plant_produce_candy_y[0][y] <= self.assigned[i][0][k])
                    self.model.add_constraint(plant2_demand[i][k] * self.plant_produce_candy_y[1][y] <= self.assigned[i][1][k])
                    self.model.add_constraint(plant3_demand[i][k] * self.plant_produce_candy_y[2][y] <= self.assigned[i][2][k])
                    self.model.add_constraint(plant4_demand[i][k] * self.plant_produce_candy_y[3][y] <= self.assigned[i][3][k])
                    
        # 2 techs, 6 supers, 3 engineers for wwach candy at each plant constraints
        for j in range(plants):
            for k in range(months):
                self.model.add_constraint(self.assigned[1][j][k] >= 2* sum(self.plant_produce_candy_y[j][y] for y in range(candys)))
                self.model.add_constraint(self.assigned[2][j][k] >= 2* sum(self.plant_produce_candy_y[j][y] for y in range(candys)))
                self.model.add_constraint(self.assigned[3][j][k] >= 2* sum(self.plant_produce_candy_y[j][y] for y in range(candys)))
                
                
        #20% constraints
        for j in range(plants):
            for k in range(months):
                for i in range(workers):
                    for y in range(candys):
                        self.model.add_constraint(0.8 *self.assigned[i][j][k-1] <= self.assignments[i][j][k])
                        self.model.add_constraint(self.assigned[i][j][k] <= 1.2* self.assignments[i][j][k-1])

            
        # Candy FF can be made at plant 1 or plant 2 (NOT BOTH)
        self.model(add_constraint(self.plant_produce_product_y[0][0] + self.plant_produce_product_y[1][0] <= 1))
                   
        #IF CC is at platn 2, then must also at plant 4
        self.model(add_constraint(self.plant_produce_product_y[1][1] <= self.plant_produce_product_y[3][1]))
        
        #TIS at either plant 1 or plant 4 (maybe both)
        self.model(add_constraint(self.plant_produce_product_y[0][2] + self.plant_produce_product_y[3][2] >= 1))


    def solve_model(self):
        solution=self.model.solve()
        
        if solution:
            j=1
            for plant in self.assigned:
                print(f"***Plant{j}***")
                i=0
                for workers in plant:
                    print(f"(worker_type{i}): ", end="")
                    for month in workers:
                        print(int(month.solution_value),end=",")
                    print()
                    i+=1
                print()
                j+=1
                
            w=1
            for plant in self.plant_produce_candy_y:
                print(f"Plant{w} " , end="")
                y=1
                for candys in plant:
                    print(int(candys.solution_value), end ="")
                    y+=1
                print()

